In [193]:
# metadata needed - assume user provides as much as possible as pre-processing step, library fills in the rest
    # instrument
    # tempo
    # key
    # percussive / harmonic / vocal --> sound_class

# so our library ultimately takes a stem and its respective metadata folder as input

# note: i think library should choose percussive stem as base 

# now we go through 4 cases: if instrument is missing, if tempo is missing, if key is missing, if percussive / harmonic / vocal is missing

In [230]:
import os
import json
import uuid
import soundfile as sf
import librosa
import glob
import random
from IPython.display import Audio
import numpy as np

In [195]:
# brid-preprocessing
# need to assign null in all cases, don't want any empty variables. assign null in "other" folders

def get_instrument_name(file_path):

    instrument_folders = {
            "-PD" : "pandeiro",
            "-TB": "tamborim",
            "-RR": "reco-reco",
            "-CX": "caixa",
            "-RP": "repique",
            "-CU": "cuica",
            "-AG": "agogo",
            "-SK": "shaker",
            "-TT": "tanta",
            "-SU": "surdo"}
    for key in instrument_folders:
        if key in file_path:
            return instrument_folders[key]
            
    return None

def get_tempo(file_path):
    
    folders = ["samba","marcha","partido alto","samba-enredo","other","capoeira"]
    
    style_folders = {
        "SA.wav" : "samba",
        "PA.wav": "partido alto",
        "CA.wav": "capoeira",
        "SE.wav": "samba-enredo",
        "MA.wav": "marcha",
        "OT.wav": "other"
    }

    style_to_tempo = {
        "samba" : 80.0,
        "partido alto" : 100.0,
        "samba-enredo" : 130.0,
        "marcha" : 120.0,
        "capoeira" : 65.0
    }
    
    style = None
    
    for suffix in style_folders:
        if suffix in file_path:
            style = style_folders[suffix]
            
    if style is None:
        return None

    if style in style_to_tempo:
        return style_to_tempo[style]

    return None

In [196]:
test_path = os.path.join("/Users/lgpietrewicz/Documents/stem_mixer/stems", "[0111] S1-PD3-04-MA.wav")

In [197]:
tempo = get_tempo(test_path)
instrument_name = get_instrument_name(test_path)
key = None
sound_class = "percussive"

In [198]:
# musdb pre-processing 

def get_type_name(file_path):
    
    type_folders = ["vocals", "drums", "bass", "other"]
    
    for name in type_folders:
        if name in file_path:
            return name
            
    return None

def get_sound_class(instrument_name):
    if instrument_name == "vocals":
        sound_class = "vocals"
    elif instrument_name == "drums":
        sound_class = "percussive"
    elif instrument_name == "bass" or instrument_name == "other":
        sound_class = "harmonic"
    else:
        sound_class = None

    return sound_class

In [199]:
tempo = None
instrument_name = get_type_name(test_path)
key = None 
sound_class = get_sound_class(test_path)

To rename a file in Python, you can use the os. rename() function from the os module. The os. rename() function takes two arguments: the current name of the file or directory (source), and the new name (destination).

In [200]:
test_path = os.path.join("/Users/lgpietrewicz/Documents/stem_mixer/stems", "Cristina Vane - So Easy - vocals.wav")

In [201]:
def metadata_extraction(path_to_stems, tempo = None, instrument_name = None, key = None, sound_class = None):
    for root, dirs, files in os.walk(path_to_stems):
        for file in files:
            file_path = os.path.join(root, file)

            if ".json" in file_path:
                continue
                
            if ".wav" or ".mp3" in file_path:
                json_name = file_path[0:len(file_path)-4]
            else:
                json_name = file_path
        
            metadata = {}
            metadata["tempo"] = tempo
            metadata["instrument_name"] = instrument_name
            metadata["key"] = key
            metadata["sound_class"] = sound_class
        
            json_file_path = os.path.join(path_to_stems, f"{json_name}.json")
            
            with open(json_file_path, "w") as json_file:
                    json.dump(metadata, json_file, indent=4)

In [202]:
home = os.getcwd()
test_path = os.path.join(home, "stems")

In [203]:
metadata_extraction(test_path, tempo = 80, sound_class = "hi")

In [204]:
# overview of process: fill in "None" values. might need to do research --> one method might be best for something like drums,
# other methods might work best for something like vocals. need to make sure all values are locked in before i start worrying about 
# generating mixtures 

In [269]:
# generating mixtures, ignoring all conditions / parameters
# send these to output folder

# want mixture outputs to be folders w n stems and mixture .wav or .mp3 files

def generate_mixture(data_home, sr, n_stems = 4, base_stem = None, percussive_to_harmonic = None):

    print(data_home)
    # make folder to store all mixtures
    mixture_folder = os.path.join(data_home, "mixtures")
    os.makedirs(mixture_folder, exist_ok = True)

    path_to_stems = os.path.join(data_home, "stems")

    all_files = [os.path.join(path_to_stems, f) for f in os.listdir(path_to_stems) if os.path.isfile(os.path.join(path_to_stems, f))]

    audio_files = []
    for file in all_files:
        if ".json" not in file:
            audio_files.append(file)
            
    if len(audio_files) < n_stems:
        raise ValueError(f"Not enough files in the directory to choose {num_files} random files.")
    
    # select random files
    random_files = random.sample(audio_files, n_stems)

    print(random_files)

    stem_audios = []
    stem_audio_lengths = []
    
    for stem in random_files:

        stem_audio, sr = librosa.load(stem, sr = sr)
        stem_audio_len = len(stem_audio)
            
        stem_audios.append(stem_audio)
        stem_audio_lengths.append(stem_audio_len)
        
        # display(Audio(stem_audio, rate=sr))

    min_length = min(stem_audio_lengths)
    min_pos = stem_audio_lengths.index(min_length)
    print(min_length, min_pos)
    mixture_audio = stem_audios[min_pos]

    truncated_stems = []
    for audio in stem_audios:
        audio = audio[:min_length]
        truncated_stems.append(audio)

    for k in range(0, len(truncated_stems)):
        if(k != min_pos): # already accounted for
            print(len(truncated_stems[k]))
            mixture_audio = mixture_audio + truncated_stems[k]

    # give each mixture a random id
    mixture_id = str(uuid.uuid4())
    individual_output_folder = os.path.join(mixture_folder, mixture_id)
    os.makedirs(individual_output_folder, exist_ok = True)

    
    sf.write(f"{individual_output_folder}/mixture.wav", mixture_audio, sr)

In [270]:
home = os.getcwd()
sr = 41500

generate_mixture(home, sr)

/Users/lgpietrewicz/Documents/stem_mixer
['/Users/lgpietrewicz/Documents/stem_mixer/stems/Bobby Nobody - Stitch Up - vocals.wav', '/Users/lgpietrewicz/Documents/stem_mixer/stems/Al James - Schoolboy Facination - drums.wav', '/Users/lgpietrewicz/Documents/stem_mixer/stems/PR - Happy Daze - bass.wav', '/Users/lgpietrewicz/Documents/stem_mixer/stems/[0273] S2-SU2-04-VPA.wav']
1149231 3
1149231
1149231
1149231
